# Project 3: Disney vs Universal, by Nadia Morgen

In [1]:
import requests
import time
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

# Step 1: Get Data

In [2]:
url = 'https://api.pushshift.io/reddit/search/submission'

Note: I had issues getting the data.  Sometimes it worked; sometimes it didn't.  First it worked for Disney, but not Universal.  Then I accidentally deleted the Disney results and couldn't get more than 100 rows after that, even though I used the same code.  Ugh.  If you get errors when you grade, it might not be my fault.

In [3]:
def pull_data(subreddit, time_now):
    
    url = 'https://api.pushshift.io//reddit/search/submission?subreddit='
    
    diz_list = []
    
    # Setting epoch time
    current_time = time_now
    
    # for loop to pull the data
    for i in range(1, 52):
        res = requests.get(url, params = {
                                                 'subreddit':subreddit,
                                                 'size':100,
                                                 'lang': True,
                                                 'before':current_time
                                            })
    
        if res.status_code == 200:
            # print i is for troubleshooting - grader might need it
            # print(i)
            
            # Set up dataframe
            df = pd.DataFrame(res.json()['data'])
            df = df.loc[:,['title','selftext', 'created_utc','subreddit']]
            diz_list.append(df)

            # Time delay            
            time.sleep(.3)
            current_time = df['created_utc'].min()

        elif 'df' in locals():
            return df
        else:
            return 'Request Failed'
        
                    
    return pd.concat(diz_list, axis= 0)

disney = pull_data('disneyparks', 1631390454)

KeyboardInterrupt: 

In [30]:
disney

In [ ]:
# Repeat for Universal
# Note to grader: I had some issues with pulling the data for universal, even though I used the same
# function as I did for Disney, but with the universal subreddit and different list name.
# For reasons unknown to me and Eric, I couln't get more than about 52 iterations, and the number
# of successful iterations differed each time I ran the exact same code.

In [ ]:
def pull_data_uni(subreddit, time_now):
    
    url = 'https://api.pushshift.io//reddit/search/submission?subreddit='
    
    univ_list = []
    
    # Setting epoch time
    current_time = time_now
    
    # for loop to pull the data
    for i in range(1, 52):
        res = requests.get(url, params = {
                                                 'subreddit':subreddit,
                                                 'size':100,
                                                 'lang': True,
                                                 'before':current_time
                                            })
    
        if res.status_code == 200:
            print(i)
            
            # Set up dataframe
            df2 = pd.DataFrame(res.json()['data'])
            df2 = df2.loc[:,['title','selftext', 'created_utc','subreddit']]
            univ_list.append(df2)

            # Time delay            
            time.sleep(.3)
            current_time = df2['created_utc'].min()

        elif 'df2' in locals():
            return df2
        else:
            return 'Request Failed'
        
                    
    return pd.concat(univ_list, axis= 0)

universal = pull_data_uni('universalstudios', 1631390454)

In [ ]:
universal

In [ ]:
# Merge two dataframes
df = pd.concat([disney, universal])

In [41]:
# It's better to load the data using this code:
df = pd.read_csv('df_clean.csv')

# Step 2:  Clean the Data

In [ ]:
# Make title and selftext lowercase

df['title'] = df.title.str.lower()
df['selftext'] = df.selftext.str.lower()

In [ ]:
# Remove stop words and giveaway words, such as "Disney"
# https://www.datasciencelearner.com/custom-stopwords-python-nlp/

nltk.download('stopwords')
#stop_words_list = stopwords.words('english')


In [ ]:
# Set custom stop words
custom = ['disney', 'universal', 
 'animal kingdom', 'magic kingdom', 'hollywood studios', 'epcot', 'anaheim', 'eurodisney', 
 'typhoon lagoon', 'blizzard beach', 'wdw', 'main street', 'disneyland', 'preferred pass', 
 'city walk', 'seasonal pass', 'power pass', 'preferred pass', 'premier pass', 'volcano bay',
 'islands of adventure', 'express pass', 'express unlimited', 'moscow', 'beijing', 'singapore',
 'mickey', 'cruise', 'incredi-pass', 'sorcerer pass', 'pirate pass', 'pixie dust pass', 
 'platinum pass', 'gold pass', 'silver pass', 'bronze pass', 'walt', 'land paris', 'tokyo land',
 'park hopper', 'studios hollywood', 'studios orlando', 'land']

In [ ]:
final_stopwords = custom

In [ ]:
# Implement stop words
nltk.download('punkt')

In [50]:
# I had a LOT of trouble pulling my data; I'm backing up everything!
df_backup = df.copy()

In [ ]:
# Verify the backup is there
df_backup.head()

In [ ]:
df.isna().sum()

In [ ]:
# Since only 21 posts have missing selftext, I'm dropping those 21
df.dropna(subset=['selftext'], inplace=True)

In [ ]:
# remove stop words

bye_stopwords = '|'.join(final_stopwords)
df['title_clean'] = df.title.replace(bye_stopwords,'', regex=True)
df['selftext_clean'] = df.selftext.replace(bye_stopwords,'', regex=True)

# https://stackoverflow.com/questions/66129517/remove-custom-stop-words-from-pandas-dataframe-not-working

In [ ]:
# Verify
df.head(15)

In [10]:
# Tokenize title and selftext

df['title_clean'] = df.apply(lambda row: nltk.word_tokenize(row['title_clean']), axis=1)
df['selftext_clean'] = df.apply(lambda row: nltk.word_tokenize(row['selftext_clean']), axis=1)

# https://stackoverflow.com/questions/52869456/how-to-use-word-tokenize-on-a-single-column-in-a-data-frame-python

In [11]:
# Verifying that data is tokenized
df.head(10)

,Unnamed: 0,title,selftext,created_utc,title_clean,selftext_clean,clean_lemon_tokens,cln_lmn_tok_title,disney_y
0,0,anybody have this 2099 expiration issue with n...,NaN,1631369894,"[[, 'anybody, ', ,, 'have, ', ,, 'this, ', ,, ...","[[, ]]",[],"['anybody', 'have', 'this', '2099', 'expiratio...",1
1,1,"i went to wdw back in april, here are some pic...",NaN,1631367728,"[[, ', i, ', ,, 'went, ', ,, 'to, ', ,, 'back,...","[[, ]]",[],"['i', 'went', 'to', 'back', 'in', 'april', ','...",1
2,2,full walkthrough of villain's grove at opening...,NaN,1631339851,"[[, 'full, ', ,, 'walkthrough, ', ,, 'of, ', ,...","[[, ]]",[],"['full', 'walkthrough', 'of', 'villain', ""'s"",...",1
3,3,one of my favorite pieces of park merchandise ...,NaN,1631339788,"[[, 'one, ', ,, 'of, ', ,, 'my, ', ,, 'favorit...","[[, ]]",[],"['one', 'of', 'my', 'favorite', 'piece', 'of',...",1
4,4,first time at avengers campus and rode incredi...,NaN,1631337412,"[[, 'first, ', ,, 'time, ', ,, 'at, ', ,, 'ave...","[[, ]]",[],"['first', 'time', 'at', 'avenger', 'campus', '...",1
5,5,university study on visiting the disney theme ...,hi! we’re a group of researchers at texas tech...,1631312443,"[[, 'university, ', ,, 'study, ', ,, 'on, ', ,...","[[, 'hi, ', ,, ', !, ', ,, 'we, ', ,, ', ’, ',...","['hi', '!', 'we', '’', 're', 'a', 'group', 'of...","['university', 'study', 'on', 'visiting', 'the...",1
6,6,university study on visiting the disney theme ...,[removed],1631311774,"[[, 'university, ', ,, 'study, ', ,, 'on, ', ,...","[[, ', [, ', ,, 'removed, ', ,, ', ], ', ]]","['[', 'removed', ']']","['university', 'study', 'on', 'visiting', 'the...",1
7,7,“we believe in our idea: a family park where p...,NaN,1631307042,"[[, ', “, ', ,, 'we, ', ,, 'believe, ', ,, 'in...","[[, ]]",[],"['“', 'we', 'believe', 'in', 'our', 'idea', ':...",1
8,8,"when you think about it, epcot forever is a pr...","it's a wonderful bunch of beloved original, cr...",1631303562,"[[, 'when, ', ,, 'you, ', ,, 'think, ', ,, 'ab...","[[, 'it, ', ,, ``, 's, '', ,, ', a, ', ,, 'won...","['it', ""'s"", 'a', 'wonderful', 'bunch', 'of', ...","['when', 'you', 'think', 'about', 'it', ',', '...",1
9,9,bring fritters back to disneyland!!! the ones ...,NaN,1631245797,"[[, 'bring, ', ,, 'fritters, ', ,, 'back, ', ,...","[[, ]]",[],"['bring', 'fritter', 'back', 'to', '!', '!', '...",1


In [ ]:
list_=[]

for row in df['selftext_clean']:
    lem_row = [lemmatizer.lemmatize(string) for string in row]
    list_.append(lem_row)
df[f'clean_lemon_tokens'] = list_


In [ ]:
list_title=[]

for row in df['title_clean']:
    lem_row = [lemmatizer.lemmatize(string) for string in row]
    list_title.append(lem_row)
df[f'cln_lmn_tok_title'] = list_title

In [ ]:
# Verify it worked
df.head(15)

In [ ]:
# Dummmify subreddit: 1 = Disney, 0 = Universal
# Note: I didn't use drop_first, because I wanted Disney to appear in the dataframe and for
# Disney to equal 1 and Universal 0, since I'm pitching to Disney

df = pd.get_dummies(df, columns=['subreddit'])

In [ ]:
# Verify it worked
df.head(1)

In [25]:
# Drop duplicate dummy column
df.drop(columns=['subreddit_universalstudios', 'Unnamed: 0'], inplace=True)

In [ ]:
df.rename({'subreddit_disneyparks':'disney_y'}, axis=1, inplace=True)

In [27]:
df.head(1)

,title,selftext,created_utc,title_clean,selftext_clean,clean_lemon_tokens,cln_lmn_tok_title,disney_y
0,anybody have this 2099 expiration issue with n...,NaN,1631369894,"[[, 'anybody, ', ,, 'have, ', ,, 'this, ', ,, ...","[[, ]]",[],"['anybody', 'have', 'this', '2099', 'expiratio...",1


In [43]:
# The selftext field has a lot more missing values than isna counted
# Copying to df2 for future analysis on selftext

df2 = df.copy()
df2.dropna(subset=['selftext'], inplace=True)
df2.drop(columns=['Unnamed: 0'], inplace=True)

In [44]:
df2.shape

(3911, 9)

In [45]:
df2.rename({'subreddit_disneyparks':'disney_y'}, axis=1, inplace=True)

In [46]:
df2.head(1)

,Unnamed: 0.1,title,selftext,created_utc,title_clean,selftext_clean,clean_lemon_tokens,cln_lmn_tok_title,disney_y
5,5,university study on visiting the disney theme ...,hi! we’re a group of researchers at texas tech...,1631312443,"['[', ""'university"", ""'"", ',', ""'study"", ""'"", ...","['[', ""'hi"", ""'"", ',', ""'"", '!', ""'"", ',', ""'w...","['hi', '!', 'we', '’', 're', 'a', 'group', 'of...","['university', 'study', 'on', 'visiting', 'the...",1


In [19]:
# Total number of disneyparks subreddit posts in complete dataframe (n = 10,177) - for title
df['disney_y'].sum()

5093

In [21]:
# Total number of universalstudios subreddit posts in complete dataframe (n = 10,177) - for title
10177 - 5093

5084

In [22]:
# Total number of disneyparks subreddit posts in small dataframe (n = 3,911) - for selftext
df2['disney_y'].sum()

1413

In [23]:
# Total number of universalstudios subreddit posts in small dataframe (n = 3,911) - for selftext
3911 - 1413

2498

In [2]:
df.to_csv('data_finally_clean.csv', index=True)
df2.to_csv('df2_clean.csv')